# Uso de callbacks

En este ejercicio, implementaremos métodos de callbacks para detener el entrenamiento cuando se cumpla una métrica específica. Esta es una función útil, ya que permitirá no completar todas las épocas cuando se alcance este umbral. Por ejemplo, si configuramos 1000 épocas y la precisión deseada ya se alcanza en la época 200, el entrenamiento se detendrá automáticamente. Veamos cómo se implementa esto en las siguientes secciones.

## Cargamos y normalizamos el conjunto de datos MNIST de moda

Al igual que en el ejercicio anterior, volveremos a utilizar el conjunto de datos Fashion MNIST.Recuerda que debemos ormalizar los valores de píxeles para ayudar a optimizar el entrenamiento.

In [1]:
import tensorflow as tf

mnist=tf.keras.datasets.fashion_mnist

(x_train, y_entrenamiento),(x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0


4422102/4422102 [==============================] - 0s 0us/step


## Creamos una clase callbacks

Puede crear un método callbacks definiendo una clase que herede la clase base [tf.keras.callbacks.Callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback). Desde allí, pudemos definir los métodos disponibles para establecer dónde se ejecutará la evaluación del método. Por ejemplo, a continuación, utilizará el método [on_epoch_end()](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback#on_epoch_end) para verificar el valor de la loss function al final de la época.

In [2]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if(logs.get('loss') < 0.6):
      self.model.stop_training = True

# Instanciamos la clase
callbacks = myCallback()

## Definimos y compilamos el modelo

A continuación, definiremos y compilaremos el modelo. La arquitectura será similar a la que se construyó en el laboratorio anterior. Luego, estableceremos el optimizador, la loss function y las métricas que usaremos para el entrenamiento.

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Entrenamos al modelo

Ahora estamos listos para entrenar el modelo. Para configurar la devolución de llamada, simplemente configure el parámetro `callbacks` en la instancia `myCallback` que declaramos anteriormente.

In [4]:
model.fit(x_train,y_entrenamiento, epochs = 15, callbacks = [callbacks])

Epoch 1/15


1875/1875 [==============================] - 17s 8ms/step - loss: 0.4756 - accuracy: 0.8299


In [5]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.4053 - accuracy: 0.8539


[0.40526899695396423, 0.8539000153541565]

Notarás que el entrenamiento no necesita completar las 10 épocas. Al evaluar el méotodo al final de la época, podemos verificar los parámetros de entrenamiento y comparar si cumplimos con el umbral establecido en la definición de la función. En este caso, simplemente se detendrá cuando la loss function caiga por debajo de `0.40` después de la época actual.

*Desafío opcional: modifica el código para que el entrenamiento se detenga cuando la métrica de precisión supere el 60 %.*